Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
# index_cols = ['shop_id', 'item_id', 'date_block_num']
index_cols = ['shop_id', 'item_id', 'date_block_num']
# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
grid[:2]

grid_vstack = np.vstack(grid)
print('grid shape:', grid_vstack.shape)
# grid_vstack

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
print('grid.head():\n', grid.head(), sep='')

#get aggregated values for (shop_id, item_id, month)
# gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'})
gb.rename(columns={'item_cnt_day':'target'}, inplace=True)
print('gb.head():\n', gb.head(2), sep='')

#fix column names
# gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
# print(gb.head())
#join aggregated data to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
print('all_data.head():\n', all_data.head(), sep='')
all_data

grid shape: (10913850, 3)
grid.head():
   shop_id  item_id  date_block_num
0       59    22154               0
1       59     2552               0
2       59     2554               0
3       59     2555               0
4       59     2564               0
gb.head():
   shop_id  item_id  date_block_num  target
0        0       30               1    31.0
1        0       31               1    11.0
all_data.head():
        shop_id  item_id  date_block_num  target
139255        0       19               0     0.0
141495        0       27               0     0.0
144968        0       28               0     0.0
142661        0       29               0     0.0
138947        0       32               0     6.0


,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0
...,...,...,...,...
10768834,59,22162,33,0.0
10769024,59,22163,33,0.0
10769690,59,22164,33,0.0
10771216,59,22166,33,0.0


In [4]:
grid_pd = pd.DataFrame(
            index=pd.MultiIndex.from_product([
                sales['shop_id'].unique(),
                sales['item_id'].unique(),
                sales['date_block_num'].unique()],
            names=index_cols)
            ).reset_index()
print('grid_pd shape:', grid_pd.shape)

all_data_pd = pd.merge(grid_pd, gb, how='left', on=index_cols).fillna(0)
print('all_data_pd shape:', all_data_pd.shape)

all_data_pd.sort_values(['date_block_num','shop_id','item_id'], inplace=True)
all_data_pd

grid_pd shape: (44486280, 3)
all_data_pd shape: (44486280, 4)


,shop_id,item_id,date_block_num,target
13195842,0,0,0,0.0
13146916,0,1,0,0.0
13188056,0,2,0,0.0
13177856,0,3,0,0.0
13195808,0,4,0,0.0
...,...,...,...,...
432479,59,22165,33,0.0
463895,59,22166,33,0.0
444447,59,22167,33,0.0
202469,59,22168,33,0.0


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [5]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()
print(item_id_target_mean.head())

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 
print('all_data:\n', all_data.head(), sep='')

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

item_id
0    0.020000
1    0.023810
2    0.019802
3    0.019802
4    0.020000
Name: target, dtype: float64
all_data:
        shop_id  item_id  date_block_num  target  item_target_enc
139255        0       19               0     0.0         0.022222
141495        0       27               0     0.0         0.056834
144968        0       28               0     0.0         0.141176
142661        0       29               0     0.0         0.037383
138947        0       32               0     6.0         1.319042
0.4830386988621699


#### Method 2

In [6]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 
print('all_data.shape = ', all_data.shape, ', all_data:\n', all_data.head(), sep='')

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print('correlation coefficient:', np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

all_data.shape = (10913850, 5), all_data:
        shop_id  item_id  date_block_num  target  item_target_enc
139255        0       19               0     0.0         0.022222
141495        0       27               0     0.0         0.056834
144968        0       28               0     0.0         0.141176
142661        0       29               0     0.0         0.037383
138947        0       32               0     6.0         1.319042
correlation coefficient: 0.4830386988621699


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [7]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [8]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold

all_data['item_target_mean'] = None
kf = KFold(n_splits=5, shuffle=False)
for train_index, val_index in kf.split(all_data):
    X_tr, X_val = all_data.iloc[train_index], all_data.iloc[val_index]
    means = X_val['item_id'].map(X_tr.groupby('item_id')['target'].mean())
    all_data['item_target_mean'].iloc[val_index] = means

# Fill NaNs with global mean
all_data['item_target_mean'].fillna(all_data['target'].mean(), inplace=True)
print('all_data.shape = ',all_data.shape, ', all_data:\n', all_data.head().to_string(), sep='')

# You will need to compute correlation like that
encoded_feature = all_data['item_target_mean'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


all_data.shape = (10913850, 6), all_data:
        shop_id  item_id  date_block_num  target  item_target_enc  item_target_mean
139255        0       19               0     0.0         0.022222          0.334273
141495        0       27               0     0.0         0.056834          0.048523
144968        0       28               0     0.0         0.141176          0.142424
142661        0       29               0     0.0         0.037383          0.030303
138947        0       32               0     6.0         1.319042          0.894020
0.41645904885340096
Current answer for task KFold_scheme is: 0.41645904885340096


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [9]:
# YOUR CODE GOES HERE

msum = all_data['item_id'].map(all_data.groupby('item_id')['target'].sum())
# print(msum.head())
mcount = all_data['item_id'].map(all_data.groupby('item_id')['target'].count())
# print(mcount.head())
all_data['item_target_loo'] = (msum - all_data['target']) / mcount
all_data

,shop_id,item_id,date_block_num,target,item_target_enc,item_target_mean,item_target_loo
139255,0,19,0,0.0,0.022222,0.334273,0.022222
141495,0,27,0,0.0,0.056834,0.048523,0.056834
144968,0,28,0,0.0,0.141176,0.142424,0.141176
142661,0,29,0,0.0,0.037383,0.030303,0.037383
138947,0,32,0,6.0,1.319042,0.894020,1.315259
...,...,...,...,...,...,...,...
10768834,59,22162,33,0.0,1.556793,0.020000,1.556793
10769024,59,22163,33,0.0,0.581395,0.334273,0.581395
10769690,59,22164,33,0.0,1.235589,0.334273,1.235589
10771216,59,22166,33,0.0,0.295918,0.349227,0.295918


In [10]:
# Fill NaNs with global mean
all_data['item_target_loo'].fillna(all_data['target'].mean(), inplace=True)
# print('all_data.shape = ', all_data.shape, ', all_data:\n', all_data.head(), sep='')

encoded_feature = all_data['item_target_loo'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.48039280661981715
Current answer for task Leave-one-out_scheme is: 0.48039280661981715


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [11]:
# YOUR CODE GOES HERE
globalmean = 0.3343
alpha = 100

item_id_target_mean = all_data.groupby('item_id')['target'].mean()
item_id_target_nrows = all_data.groupby('item_id')['target'].count()
item_id_target_smooth = ((item_id_target_mean*item_id_target_nrows + globalmean*alpha)/
                         (item_id_target_nrows + alpha))
all_data['item_target_smooth'] = all_data['item_id'].map(item_id_target_smooth)
all_data

,shop_id,item_id,date_block_num,target,item_target_enc,item_target_mean,item_target_loo,item_target_smooth
139255,0,19,0,0.0,0.022222,0.334273,0.022222,0.237448
141495,0,27,0,0.0,0.056834,0.048523,0.056834,0.089905
144968,0,28,0,0.0,0.141176,0.142424,0.141176,0.168964
142661,0,29,0,0.0,0.037383,0.030303,0.037383,0.107910
138947,0,32,0,6.0,1.319042,0.894020,1.315259,1.260635
...,...,...,...,...,...,...,...,...
10768834,59,22162,33,0.0,1.556793,0.020000,1.556793,1.334117
10769024,59,22163,33,0.0,0.581395,0.334273,0.581395,0.473493
10769690,59,22164,33,0.0,1.235589,0.334273,1.235589,1.054970
10771216,59,22166,33,0.0,0.295918,0.349227,0.295918,0.298926


In [12]:
encoded_feature = all_data['item_target_smooth'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.4818198797097264
Current answer for task Smoothing_scheme is: 0.4818198797097264


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [28]:
# YOUR CODE GOES HERE

cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']
cumcnt = all_data.groupby('item_id').cumcount()
all_data['item_target_emscheme'] = cumsum/cumcnt
all_data['item_target_emscheme'].fillna(all_data['target'].mean(), inplace=True)
all_data

,shop_id,item_id,date_block_num,target,item_target_enc,item_target_mean,item_target_loo,item_target_smooth,item_target_emscheme
139255,0,19,0,0.0,0.022222,0.334273,0.022222,0.237448,0.334273
141495,0,27,0,0.0,0.056834,0.048523,0.056834,0.089905,0.334273
144968,0,28,0,0.0,0.141176,0.142424,0.141176,0.168964,0.334273
142661,0,29,0,0.0,0.037383,0.030303,0.037383,0.107910,0.334273
138947,0,32,0,6.0,1.319042,0.894020,1.315259,1.260635,0.334273
...,...,...,...,...,...,...,...,...,...
10768834,59,22162,33,0.0,1.556793,0.020000,1.556793,1.334117,1.560268
10769024,59,22163,33,0.0,0.581395,0.334273,0.581395,0.473493,0.585938
10769690,59,22164,33,0.0,1.235589,0.334273,1.235589,1.054970,1.238693
10771216,59,22166,33,0.0,0.295918,0.349227,0.295918,0.298926,0.296170


In [29]:
# del all_data['item_target_mscheme']

In [30]:
encoded_feature = all_data['item_target_emscheme'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.5025245189907681
Current answer for task Expanding_mean_scheme is: 0.5025245189907681


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [ ]:
STUDENT_EMAIL = # EMAIL HERE
STUDENT_TOKEN = # TOKEN HERE
grader.status()

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)